In [24]:
import xlrd
import sys
sys.path.append('..')
import pandas as pd
from utils import *
from db_utils import *
import re
import argparse
from IPython.display import display

In [34]:
def change_date(date_str):
        date,month,year = date_str.split('/')
        return '-'.join(['20'+year,month,date])

def get_universal_transaction_dict(file_path):
        workbook = xlrd.open_workbook(file_path)
        df = pd.DataFrame(columns=['Date','Transaction Remarks','Reference Number','Value Date','Withdrawal Amount','Deposit Amount','Balance'])
        sheet = workbook.sheet_by_index(0)

        #read table starting from 22th row
        for rx in range(22,sheet.nrows):
            df.loc[len(df)] = [x.value for x in sheet.row(rx)]
            if df.iloc[len(df) -1, 4] == "":
                    df.iloc[len(df) -1, 4] = 0.0
            if df.iloc[len(df) -1, 5] == "":
                    df.iloc[len(df) -1, 5] = 0.0

        #drop redundant column cheque no.
        df = df.drop(['Reference Number'],axis=1)
        # df = page_break_merge_statement(df)

                #remove bottom metainfo
        df.drop(df.tail(26).index,
                inplace = True)

        final_list = []
        display(df.tail())
        for index,row in df.iterrows():
            try:
                raw_doc = row.to_dict()
                final_list.append({
                    'Date': change_date(raw_doc['Value Date']),
                    'Amount': raw_doc['Withdrawal Amount']+raw_doc['Deposit Amount'],
                    'TransactionType': determine_transaction_type(raw_doc),
                    'Balance':raw_doc['Balance'],
                    'Remarks_parsed':raw_doc['Transaction Remarks'],
                    'Remarks_raw':raw_doc['Transaction Remarks'],
                    'source':'hdfc_bank'
                })
            except Exception as e:
                print(e,raw_doc)
        return final_list

In [35]:
final_transaction = []

In [36]:
final_transaction.extend(get_universal_transaction_dict('../excels/hdfc_2022.xls'))
# connection = create_connection('global_statement.db')
# create_table_icici_statement('ALL_TRANSACTION', connection)
# insert_many('ALL_TRANSACTION', transaction_list, connection)
# cursor = fetch_data(connection, 'ALL_TRANSACTION')
# for row in cursor.fetchall():
#     print(row)


,Date,Transaction Remarks,Value Date,Withdrawal Amount,Deposit Amount,Balance
135,28/12/22,UPI-NIRAJ AGARWAL-PAYTMQR28100505010137Y363PE1...,28/12/22,471.0,0.0,34990.74
136,28/12/22,UPI-RAMESH V-PAYTMQR28100505010113J0HGPP77JW@P...,28/12/22,104.0,0.0,34886.74
137,29/12/22,UPI-RAMESH V-PAYTMQR28100505010113J0HGPP77JW@P...,29/12/22,30.0,0.0,34856.74
138,29/12/22,UPI-NIRAJ AGARWAL-PAYTMQR28100505010137Y363PE1...,29/12/22,96.0,0.0,34760.74
139,29/12/22,UPI-PRITESH SHANTILAL KA-PRITESHKAMANI.CA@OKHD...,29/12/22,100.0,0.0,34660.74


In [37]:
len(final_transaction)

140

In [47]:
final_transaction_remarks = [i['Remarks_raw'] for i in final_transaction]

In [48]:
len(final_transaction_remarks)

140

In [49]:
unique_remarks = list(set(final_transaction_remarks))

In [50]:
clean_remarks = []
for i in unique_remarks:
    if '-' not in i:
        print(i)
    else:
        clean_remarks.append(i)

POS 419188XXXXXX5234 SWIGGY INSTAMART
POS 419188XXXXXX5234 BHP*ROKLAS ENTER
POS 419188XXXXXX5234 TRIAKSHAAENTERPR
POS 419188XXXXXX5234 PVR LIMITED
POS 419188XXXXXX5234 NOVOTEL KOCHI IN
POS 419188XXXXXX5234 LULU INTERNATION
POS 419188XXXXXX5234 NAGARJUNA ANDHRA
POS 419188XXXXXX5234 ANNA VILAAS
POS 419188XXXXXX5234 PUTHOORAM BEACH
POS 419188XXXXXX5234 HM COM
POS 419188XXXXXX5234 SWIGGY
POS 419188XXXXXX5234 TW COFFEE  HSR 2
CREDIT INTEREST CAPITALISED
POS 419188XXXXXX5234 CRAFT TREE
POS 419188XXXXXX5234 CHIRAG HOSPITAL,
POS 419188XXXXXX5234 PAUL MURICKEN
POS 419188XXXXXX5234 KASHI ART CAFE
POS 419188XXXXXX5234 VENKATESWARA AND
POS 419188XXXXXX5234 TATA STARBUCKS P
PRIN AND INT AUTO_REDEEM 50300580397399
POS 419188XXXXXX5234 THE BOMBAY STORE


In [52]:
final = []
for i in clean_remarks:
    final_str = ''
    sub_str = i.split('-')
    sub_str = [re.sub(r"\s+", '', j) for j in sub_str]
    final_str = '  '.join(sub_str)
    final.append(final_str)

In [51]:
clean_remarks

['UPI-SWIGGYINSTAMART-SWIGGYINSTAMART@YESPAY-YESB0YESUPI-230820373302-PAY FOR MERCHANT',
 'UPI-UBER INDIA SYSTEMS P-UBERRIDES@HDFCBANK-HDFC0000499-230952239188-UBERRIDE',
 'UPI-RAMESH V-PAYTMQR28100505010113J0HGPP77JW@PAYTM-PYTM0123456-231909855741-VEGETABLE',
 'UPI-HENNES N MAURITZ-HENNESNMAURITZ.76042179@HDFCBANK-HDFC0004809-231999894639-COLLECT',
 'UPI-UBER INDIA SYSTEMS P-UBERRIDES@HDFCBANK-HDFC0000499-234535110695-UBERRIDE',
 'UPI-DIPENDRA KUMAR TAPAR-DIPUKESAR@OKHDFCBANK-INDB0000008-228636676494-PRANAV',
 'UPI-SWIGGYINSTAMART-SWIGGYINSTAMART@YESPAY-YESB0YESUPI-234024514063-PAY FOR MERCHANT',
 'UPI-SWIGGY-SWIGGYSTORES@ICICI-ICIC0DC0099-231884842367-UPI',
 'UPI-MEDWORLD BENGALURU-GPAY-11214189068@OKBIZAXIS-UTIB0000000-236109785173-MEDWALL',
 'UPI-SWIGGY-SWIGGYSTORES@ICICI-ICIC0DC0099-232797792965-UPI',
 'UPI-BHARATPEMERCHANT-BHARATPE.9041922461@ICICI-ICIC0DC0099-231909594978-VERIFIED MERCHANT',
 'UPI-SWIGGY-SWIGGY@YESPAY-YESB0YESUPI-232439479286-PAY FOR MERCHANT',
 'UPI-GIRBAN DEB-

In [53]:
with open('../datasets/hdfc_2022.txt','w') as f:
    for i in final:
        f.write(i+'\n')